In [1]:
import pandas as pd
from helpers import *
import random

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
file_name = "../../input-data/queries.csv"
queries_df = pd.read_csv(file_name)
print(queries_df.info())
queries_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config_id  45 non-null     object
 1   config     45 non-null     object
 2   city       45 non-null     object
 3   query_v    45 non-null     object
dtypes: object(4)
memory usage: 1.5+ KB
None


,config_id,config,city,query_v
0,c_p_143_pop_high_hard,"{'p_id': 'p_143', 'persona': 'A former printin...","['Zurich', 'Warsaw', 'Vienna', 'Valencia', 'To...","""Recommend European cities for a high-budget t..."
1,c_p_15_pop_high_hard,"{'p_id': 'p_15', 'persona': 'A patient who see...","['Zurich', 'Vienna', 'Stuttgart', 'Stockholm',...","""High budget European cities to visit in Decem..."
2,c_p_5_pop_high_hard,"{'p_id': 'p_5', 'persona': 'an average Ukraini...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""Popular and expensive European cities to visi..."
3,c_p_155_pop_high_hard,"{'p_id': 'p_155', 'persona': 'a digital transf...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...","""European city breaks in September with high-e..."
4,c_p_116_pop_high_hard,"{'p_id': 'p_116', 'persona': 'An entrepreneuri...","['Zurich', 'Warsaw', 'Valencia', 'Toulouse', '...",High budget European city breaks in October wi...


In [4]:
kb = pd.read_csv("../../input-data/kb/eu-cities-database.csv")[["city", "popularity"]]
kb.drop_duplicates(inplace=True, ignore_index=True)
print(len(kb))
kb.head()

200


,city,popularity
0,Aalborg,Medium
1,Adana,Low
2,Adiyaman,Low
3,Agri,Low
4,Amsterdam,High


In [5]:
def generate_randomPop_recs(k=10, seed=42, kb: pd.DataFrame = None, pop_category = "High"):
    if kb is None:
        raise ValueError("kb must be provided.")

    if k > len(kb):
        raise ValueError(f"k ({k}) cannot be greater than the number of available cities ({len(kb)}).")

    # filter by pop_category
    kb = kb.loc[kb['popularity'] == pop_category]
    kb_cities = kb["city"].unique().tolist()

    random.seed(seed)
    return random.sample(kb_cities, k)

In [6]:
queries = queries_df['query_v'].tolist()
random_cities_results = []
SEEDS = [
    42, 7, 19, 83, 101, 256, 999, 123, 987, 65,
    300, 14, 29, 77, 55, 321, 444, 678, 890, 12,
    34, 56, 78, 90, 135, 246, 357, 468, 579, 680,
    791, 802, 913, 111, 222, 333, 4444, 5555, 6666,
    7777, 8888, 9999, 2025, 3141, 2718, 1618, 9090
]
POP_CATEGORIES = ["Low", "Medium", "High"]

def get_random_pop_category(seed):
    random.seed(seed)
    return random.choice(POP_CATEGORIES)

    
for (q, seed) in zip(queries, SEEDS):
    pop_category = get_random_pop_category(seed)
    random_cities_list = generate_randomPop_recs(k=10, seed=seed, kb=kb, pop_category = pop_category)
    random_cities_results.append(random_cities_list)
queries_df["randomPop_nonInformed"] = random_cities_results

In [7]:

queries_df.to_csv("../../data/conv-trs/multi-agent/results/non-llm-baselines/randomPop_baseline.csv", index=False)
queries_df[["query_v", "randomPop_nonInformed"]].head()


,query_v,randomPop_nonInformed
0,"""Recommend European cities for a high-budget t...","[Copenhagen, Baku, Milan, London, Krasnodar, D..."
1,"""High budget European cities to visit in Decem...","[Penza, Erfurt, Kirov, Zilina, Bari, Belgorod,..."
2,"""Popular and expensive European cities to visi...","[Belgrade, Warsaw, Cork, Volgograd, Kaliningra..."
3,"""European city breaks in September with high-e...","[Zaragoza, Timisoara, Bern, Brno, Kiel, Astrak..."
4,High budget European city breaks in October wi...,"[Izmir, Poznan, Tbilisi, Bergen, Vilnius, Krak..."
